In [4]:
import nltk
import time
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
from transformers import pipeline
import torch
from nltk.tokenize import word_tokenize
from sklearn.metrics import mean_squared_error, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
from sklearn.svm import SVR, SVC
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from sklearn.decomposition import PCA
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

print('Job done-')

Job done-


In [5]:
import nltk
import time
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from sklearn.svm import SVR
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
import string
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import ElasticNet

from sklearn.preprocessing import OneHotEncoder

#### 1. 데이터 준비

In [6]:
total_file_path = "data/dataset.txt"
total_documents = []
total_document = []

with open(total_file_path, 'r', encoding='utf-8') as file:
    # Postmortum 읽기
    for line in file:
        # Document separation
        if line == '\n':
            total_documents.append('\n'.join(total_document))
            total_document = []

        # Add to document
        total_document.append(line)

    # For last game document
    total_documents.append('\n'.join(total_document))

total_documents = np.array(total_documents).reshape(-1, 1)
    
# 목표값 붙이기
file_path = 'data/classification.txt'
class_list = []

class_size = 28   ### 29 -> 28
with open(file_path, 'r') as file:
    for line in file:
        class_type = str(line.strip())
        class_list.append(class_type)
        
ohe = OneHotEncoder(sparse=False)
le = LabelEncoder()
le_label = le.fit_transform(np.array(class_list)).reshape(-1, 1)
ohe_label = ohe.fit_transform(np.array(class_list).reshape(-1, 1))

label_size = le_label.shape[1] + ohe_label.shape[1]
data_index = np.arange(1, total_documents.shape[0] + 1).reshape(-1, 1)
data = np.hstack((data_index, total_documents, le_label, ohe_label))

print('jobs finished b')

# 데이터셋 분할
test_ratio = 0.2
index_size = 1
X_train, X_test, y_train, y_test = train_test_split(data[:, :-label_size],
                                                    data[:, -label_size:],
                                                    test_size=test_ratio,
                                                    stratify=data[:, -label_size:-label_size + le_label.shape[1]],
                                                    random_state=0)

# 자료형 변환
X_train_index, X_test_index = X_train[:, :index_size].astype(int), X_test[:, :index_size].astype(int)
X_train, X_test = X_train[:, index_size:], X_test[:, index_size:]
X_train = X_train.flatten()
X_test = X_test.flatten()
y_train_ohe = y_train[:, le_label.shape[1]:].astype(np.float64)
y_test_ohe = y_test[:, le_label.shape[1]:].astype(np.float64)
y_train_le = y_train[:, :le_label.shape[1]].astype(np.float64)
y_test_le = y_test[:, :le_label.shape[1]].astype(np.float64)

# Result Preparation
result_table = pd.DataFrame(columns=['index', 'type', 'model', 'truth', 'prediction', 'f1', 'auc'])

jobs finished b


/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


#### 2. 워드 임베딩

In [7]:
# TF-IDF 학습
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(X_train).toarray()
print(f'{X_train.shape=}')

# 검증 데이터 워드 임베딩
X_test = tfidf_vectorizer.transform(X_test).toarray()
print(f'{X_test.shape=}')

X_train.shape=(534, 4513)
X_test.shape=(134, 4513)


#### 3. 차원 축소

In [8]:
pca_var_ratio = 0.95
pca = PCA(n_components=pca_var_ratio)
original_dimension = X_train.shape[1]
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

print(f'Reduced Dimension: {original_dimension} -> {X_train.shape[1]}')

Reduced Dimension: 4513 -> 440


#### 4. Model Train

##### 4-3. rf

In [9]:
#X_train, X_test, y_train, y_test = train_test_split(data[:, :vector_size + label_size],
 #                                                   data[:, vector_size + label_size],
  #                                                  test_size=test_ratio,
   #                                                 stratify=data[:, vector_size + label_size],
    #                                                random_state=0)


# 필요한 라이브러리 임포트
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics import accuracy_score

# Random Forest 모델 생성 및 학습

rf_model = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
rf_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = rf_model.predict(X_test)

# 정확도 및 분류 보고서 출력
accuracy = accuracy_score(y_test, y_pred)
auc_score = roc_auc_score(y_test, y_pred, multi_class='ovr')  # 'ovr'은 One-vs-Rest 방식을 사용함
print(f'AUC: {auc_score:0.3f}')
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

ValueError: multiclass-multioutput is not supported

##### 4-4. xgb

In [9]:
param_space = {
    'learning_rate': (0.01, 1.0, 'log-uniform'),
    'max_depth': (1, 50),
    'n_estimators': (50, 200),
    'min_child_weight': (1, 10),
    'subsample': (0.1, 1.0, 'uniform'),
    'gamma': (0.1, 5.0),
    'colsample_bytree': (0.1, 1.0, 'uniform'),
    'alpha': (0.0, 1.0, 'uniform'),
    'reg_lambda': (0.0, 1.0, 'uniform')
}

# 하이퍼 파라미터 튜닝
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')
opt = BayesSearchCV(xgb_model, param_space, n_iter=50, cv=10, n_jobs=-1, n_points=8, verbose=0)

timer = time.time()
opt.fit(X_train, y_train)
print(f'Elapsed time for xgb optimization: {(time.time() - timer)/60:0.2f} min')
print("최적의 하이퍼파라미터:", opt.best_params_)

# XGB 학습
timer = time.time()
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', **opt.best_params_)
xgb_model.fit(X_train, y_train)
print(f'Elapsed time for xgb training: {(time.time() - timer)/60:0.2f} min')

# XGB 학습데이터 추론
print('[XGB 학습 데이터]')
train_prediction = xgb_model.predict(X_train)
print(f'{train_prediction=}')
rmse_list = [np.abs(y_train[i] - train_prediction[i]) for i in range(len(y_train))]
rmse = np.sqrt(mean_squared_error(y_train, train_prediction))
rmse_std = np.std(rmse_list)
print(f'result list: {rmse_list}')
print(f'result: {rmse:0.3f} (std: {rmse_std:0.3f})')
for index in range(train_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_train_index[index], 'train', 'XGB', y_train[index], train_prediction[index], rmse_list[index]]

# XGB 추론
print('[XGB 검증 데이터]')
test_prediction = xgb_model.predict(X_test)
print(f'{test_prediction=}')
rmse_list = [np.abs(y_test[i] - test_prediction[i]) for i in range(len(y_test))]
rmse = np.sqrt(mean_squared_error(y_test, test_prediction))
rmse_std = np.std(rmse_list)
print(f'result list: {rmse_list}')
print(f'result: {rmse:0.3f} (std: {rmse_std:0.3f})')
for index in range(test_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_test_index[index], 'test', 'XGB', y_test[index], test_prediction[index], rmse_list[index]]

Elapsed time for xgb optimization: 0.43 min
최적의 하이퍼파라미터: OrderedDict([('alpha', 1.0), ('colsample_bytree', 1.0), ('gamma', 0.10966739378542173), ('learning_rate', 1.0), ('max_depth', 1), ('min_child_weight', 10), ('n_estimators', 50), ('reg_lambda', 0.0), ('subsample', 0.10066623913222518)])
Elapsed time for xgb training: 0.00 min
[XGB 학습 데이터]
train_prediction=array([85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8,
       85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8,
       85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8,
       85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8,
       85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8, 85.8,
       85.8, 85.8, 85.8, 85.8, 85.8], dtype=float32)
result list: [4.8000030517578125, 4.1999969482421875, 5.1999969482421875, 4.1999969482421875, 6.1999969482421875, 8.800003051757812, 11.199996948242188, 4.8000030517578125, 7.1999969482421875, 11.199996948242188, 8.1999

In [10]:
result_table.to_excel('result/result_table_tfidf.xlsx', index=False)

pd.pivot_table(result_table,
               values=['rmse'],
               index=['type', 'model'],
               aggfunc=['mean', 'std'])

mean        std
                  rmse       rmse
type  model                      
test  LR     10.209833  13.005579
      RF     10.317801  13.048011
      SVR     9.591844  14.367333
      XGB     9.826665  13.576634
train LR      8.016582   6.538506
      RF      9.400648   7.310793
      SVR     8.541950   8.566745
      XGB     9.226666   7.609225